# SnapShot preprocessing:
In this notebook we get the MATSim snapShot output as the input. MATSim gives us the user location in constant intervals. It does not report anything when the user is not traveling. So to make this data as much as possible closer to the actual periodic location update, we have to fill in the empty records. also we initially get the snapshots every 30 seconds. For other polling intervals we only reduce the accuracy. Also for the spatial aggregation to the level os TAZ zones, we use arcGIS to extract the associated TAZ of each location record from the snapShot file. The output of arcGIS is the input here.

#### adding required packages

In [1]:
import pandas as pd
import time
from math import floor
import numpy as np
import pickle
import requests
import concurrent.futures

#### specifying the saving location 

In [2]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

#### preparing the output of arcGIS for completion

The output of GIS misses the locations outside of Amsterdam. Therefore, we complete the data by considering their TAZ code `0`.

In [130]:
precompletion = pd.read_csv('{a}GISoutput_PreCompletion.CSV'.format(a=savingLoc),usecols=['mzr_id', 'VEHICLE','TIME','EASTING','NORTHING'])
precompletion = precompletion.sort_values(by=["VEHICLE","TIME"])
precompletion = precompletion.reset_index(drop=True)
with open('{a}snapShot_allowedUsers.pickle'.format(a=savingLoc),'rb') as handle:
    MATSimOutput = pickle.load(handle)
MATSimOutput=MATSimOutput.reset_index(drop=False)
print(len(MATSimOutput)-len(precompletion)," records are missing that we refill them in our snapShot. ")


7481  records are missing that we refill them in our snapShot. 


In [42]:
snapData = pd.merge(precompletion, MATSimOutput, how='right', on=['VEHICLE','TIME'])
(snapData.mzr_id[snapData.mzr_id.isna()]) = 0
snapData = snapData.loc[:,['VEHICLE','TIME','EASTING_y','NORTHING_y','mzr_id']]
snapData.columns = ['VEHICLE', 'TIME', 'EASTING', 'NORTHING', 'mzr_id']
snapData = snapData.sort_values(by = ['VEHICLE', 'TIME'])
with open('{a}finalInputPython.pickle'.format(a=savingLoc),'wb') as handle:
    pickle.dump(snapData, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Now, it is time to generate the complete snapShot data for every 30 seconds which represents the base data set even for generating other snapshots with different polling intervals (we resample from this data based on the specified polling interval).

In [5]:
with open('{a}finalInputPython.pickle'.format(a=savingLoc),'rb') as handle:
    snapData = pickle.load(handle)
snapData.reset_index(drop=True,inplace=True)
snapData.head()

,VEHICLE,TIME,EASTING,NORTHING,mzr_id
0,1,23400,632364.770972,5.816900e+06,7065.0
1,1,23430,632279.680941,5.816846e+06,5329.0
2,1,23460,632234.315601,5.816431e+06,5329.0
3,1,23490,632200.291596,5.816119e+06,5329.0
4,1,23520,632209.756236,5.815776e+06,5329.0


In [ ]:
#we time the process
startTime = time.time()

userGroups = (snapData.groupby(["VEHICLE"]))
concatData = {}
IDs = list(userGroups.groups.keys())[0:len(userGroups.groups.keys())]
for i,ID in enumerate(IDs): #userGroups.groups.keys()
    print(ID,": ",time.time() - startTime)
    records=userGroups.get_group(ID)
    records.TIME = pd.to_timedelta(records.TIME, unit="s")
    records.set_index(["TIME"],inplace=True)
    # print(kk.tail())
    records.loc[records.index[0]+pd.to_timedelta('24:00:00')]=records.iloc[0,:]
    records.sort_index(inplace=True)
    records = records.resample('30S').fillna("pad")
    records.drop([records.index[0]+pd.to_timedelta('24:00:00')],axis=0,inplace=True)
    concatData[ID] = records
# with open("completePLUdata_30sec_dict.pickle".format(a=savingLoc),"wb") as handle:
#           pickle.dump(concatData,handle, protocol=pickle.HIGHEST_PROTOCOL )
# snapDataNew = pd.DataFrame()
# for ID in concatData.keys():
#     snapDataNew = snapDataNew.append(concatData[ID])
# with open("completePLUdata_30sec.pickle".format(a=savingLoc),"wb") as handle:
#           pickle.dump(snapDataNew,handle, protocol=pickle.HIGHEST_PROTOCOL )
print((time.time() - startTime)//60,'minutes')

1 :  0.316180944442749
6 :  0.40990757942199707
10 :  0.41392016410827637
14 :  0.4189107418060303
16 :  0.4228963851928711
18 :  0.4278995990753174
20 :  0.4328758716583252
23 :  0.43785738945007324
26 :  0.4418466091156006
30 :  0.44684934616088867
37 :  0.450822114944458
41 :  0.4558093547821045
49 :  0.45979881286621094
75 :  0.4637877941131592
78 :  0.4687516689300537
80 :  0.47376084327697754
85 :  0.4777507781982422
89 :  0.48273706436157227
93 :  0.4877309799194336
95 :  0.4917128086090088
98 :  0.49669957160949707
104 :  0.5006911754608154
107 :  0.5066742897033691
114 :  0.5126650333404541
116 :  0.5176208019256592
119 :  0.5236368179321289
120 :  0.5285913944244385
131 :  0.5336017608642578
134 :  0.5385885238647461
135 :  0.5425779819488525
138 :  0.5475647449493408
139 :  0.5525510311126709
143 :  0.5565402507781982
147 :  0.5605306625366211
149 :  0.5655229091644287
155 :  0.5695056915283203
157 :  0.5734994411468506
158 :  0.5784816741943359
170 :  0.5824708938598633
173

1273 :  2.007673978805542
1275 :  2.0126607418060303
1278 :  2.018646478652954
1284 :  2.0256574153900146
1287 :  2.0326106548309326
1291 :  2.0376293659210205
1294 :  2.042614221572876
1295 :  2.046595811843872
1300 :  2.0515823364257812
1305 :  2.0565686225891113
1313 :  2.0605645179748535
1315 :  2.064547300338745
1316 :  2.0695338249206543
1318 :  2.0735230445861816
1319 :  2.0775136947631836
1322 :  2.081502914428711
1328 :  2.0864882469177246
1329 :  2.0904924869537354
1339 :  2.0944676399230957
1341 :  2.098456621170044
1342 :  2.1034436225891113
1346 :  2.1074326038360596
1348 :  2.111438751220703
1349 :  2.1154110431671143
1350 :  2.119403123855591
1351 :  2.124387502670288
1358 :  2.1283774375915527
1373 :  2.13336443901062
1377 :  2.1373531818389893
1382 :  2.141343355178833
1389 :  2.1463043689727783
1399 :  2.1513164043426514
1400 :  2.1553049087524414
1401 :  2.1592938899993896
1404 :  2.164280891418457
1407 :  2.1682705879211426
1411 :  2.1722607612609863
1415 :  2.17624

2432 :  3.503885269165039
2437 :  3.5088398456573486
2441 :  3.5148487091064453
2443 :  3.5188145637512207
2445 :  3.5247981548309326
2447 :  3.528787136077881
2456 :  3.53377366065979
2458 :  3.5387609004974365
2471 :  3.543776273727417
2473 :  3.5487656593322754
2475 :  3.553753137588501
2476 :  3.557741641998291
2477 :  3.561723470687866
2478 :  3.5667104721069336
2479 :  3.5707004070281982
2483 :  3.574688673019409
2492 :  3.578678607940674
2493 :  3.583665609359741
2494 :  3.587653875350952
2496 :  3.5916428565979004
2498 :  3.5956337451934814
2499 :  3.6006200313568115
2504 :  3.6046085357666016
2506 :  3.609595775604248
2509 :  3.6135847568511963
2510 :  3.617575168609619
2513 :  3.6225743293762207
2515 :  3.6275634765625
2520 :  3.631545066833496
2521 :  3.636537790298462
2522 :  3.6405136585235596
2523 :  3.6454997062683105
2524 :  3.6504862308502197
2526 :  3.6544761657714844
2528 :  3.6584653854370117
2530 :  3.6634557247161865
2534 :  3.667447566986084
2538 :  3.67143034934

3657 :  5.002880334854126
3660 :  5.006855010986328
3664 :  5.0118324756622314
3666 :  5.016819000244141
3676 :  5.02180552482605
3687 :  5.025794744491577
3691 :  5.0307817459106445
3692 :  5.035768747329712
3699 :  5.040756940841675
3702 :  5.046773195266724
3706 :  5.050760507583618
3711 :  5.055746555328369
3713 :  5.0597288608551025
3714 :  5.06471586227417
3723 :  5.0687055587768555
3732 :  5.0726940631866455
3735 :  5.076699495315552
3744 :  5.081670045852661
3745 :  5.085660219192505
3752 :  5.089648962020874
3753 :  5.093637943267822
3756 :  5.098624229431152
3765 :  5.102613687515259
3769 :  5.1096038818359375
3772 :  5.114590644836426
3783 :  5.11858606338501
3785 :  5.122560739517212
3794 :  5.12654972076416
3795 :  5.131536483764648
3796 :  5.135526180267334
3799 :  5.139515399932861
3805 :  5.143504619598389
3808 :  5.148491621017456
3810 :  5.153478145599365
3811 :  5.157467603683472
3814 :  5.162454128265381
3818 :  5.166443586349487
3821 :  5.170432806015015
3823 :  5.

4979 :  6.5019690990448
4980 :  6.50695538520813
4983 :  6.511948823928833
4985 :  6.5169360637664795
4989 :  6.521891832351685
4990 :  6.525881052017212
5003 :  6.530866861343384
5005 :  6.535886526107788
5007 :  6.540872812271118
5012 :  6.545827627182007
5018 :  6.551835775375366
5024 :  6.5558247566223145
5025 :  6.560811758041382
5033 :  6.56480073928833
5035 :  6.568791151046753
5036 :  6.5737833976745605
5038 :  6.577767848968506
5042 :  6.581764459609985
5044 :  6.585744857788086
5045 :  6.590731859207153
5053 :  6.594723224639893
5055 :  6.598710298538208
5056 :  6.6026997566223145
5060 :  6.607686758041382
5062 :  6.61167573928833
5064 :  6.615665435791016
5066 :  6.619654655456543
5068 :  6.624642610549927
5070 :  6.628631830215454
5072 :  6.632619857788086
5074 :  6.637607097625732
5078 :  6.641666889190674
5084 :  6.645585775375366
5089 :  6.650581121444702
5096 :  6.655559539794922
5097 :  6.65954852104187
5098 :  6.664535999298096
5103 :  6.668524265289307
5107 :  6.6725

...

163058 :  130.47016716003418
163059 :  130.47914266586304
163061 :  130.4851279258728
163063 :  130.49111342430115
163071 :  130.49809312820435
163074 :  130.50308108329773
163075 :  130.51006197929382
163078 :  130.51504707336426
163087 :  130.5210337638855
163089 :  130.5270164012909
163093 :  130.53299975395203
163096 :  130.53898453712463
163097 :  130.54497003555298
163101 :  130.55095291137695
163104 :  130.55593872070312
163109 :  130.56192302703857
163114 :  130.56791162490845
163118 :  130.57389116287231
163125 :  130.57987570762634
163127 :  130.58585929870605
163128 :  130.59084701538086
163129 :  130.59679627418518
163130 :  130.60278177261353
163132 :  130.60876536369324
163136 :  130.61478304862976
163139 :  130.620765209198
163142 :  130.6267499923706
163145 :  130.6327362060547
163150 :  130.63871955871582
163155 :  130.64370489120483
163156 :  130.65068650245667
163162 :  130.65567207336426
163163 :  130.66262078285217
163167 :  130.6686360836029
163171 :  130.67462635

164157 :  132.34878420829773
164158 :  132.35479760169983
164162 :  132.36078190803528
164163 :  132.36676788330078
164166 :  132.37275338172913
164175 :  132.3787341117859
164178 :  132.38471794128418
164182 :  132.39069938659668
164183 :  132.39768362045288
164186 :  132.40367150306702
164187 :  132.4096553325653
164189 :  132.4156367778778
164191 :  132.42162013053894
164193 :  132.42857098579407
164195 :  132.43458652496338
164199 :  132.44056963920593
164200 :  132.44656443595886
164201 :  132.45253682136536
164210 :  132.45852136611938
164215 :  132.46450519561768
164216 :  132.47048664093018
164217 :  132.47547793388367
164219 :  132.4824571609497
164222 :  132.488440990448
164224 :  132.49442291259766
164226 :  132.50040984153748
164227 :  132.50639367103577
164229 :  132.5123462677002
164231 :  132.51836371421814
164233 :  132.52431535720825
164234 :  132.53029918670654
164237 :  132.53727865219116
164240 :  132.54326272010803
164241 :  132.54924702644348
164243 :  132.5552656

165051 :  134.22781229019165
165054 :  134.2338273525238
165056 :  134.23981046676636
165062 :  134.24579238891602
165063 :  134.2517762184143
165064 :  134.25776290893555
165070 :  134.26374673843384
165071 :  134.26969861984253
165072 :  134.27571296691895
165074 :  134.28169894218445
165077 :  134.28768515586853
165081 :  134.29463124275208
165090 :  134.29964923858643
165091 :  134.30563497543335
165092 :  134.31158566474915
165093 :  134.31760239601135
165095 :  134.32358622550964
165097 :  134.32957100868225
165110 :  134.33555507659912
165120 :  134.341539144516
165121 :  134.34749102592468
165127 :  134.35350704193115
165141 :  134.36052560806274
165142 :  134.36651062965393
165170 :  134.37249445915222
165179 :  134.37847995758057
165182 :  134.3834583759308
165183 :  134.38944721221924
165193 :  134.39540028572083
165196 :  134.40138339996338
165205 :  134.40739679336548
165206 :  134.41438031196594
165209 :  134.42036628723145
165214 :  134.42635035514832
165220 :  134.43233

166204 :  136.10403442382812
166207 :  136.11002039909363
166209 :  136.11600351333618
166216 :  136.12198615074158
166218 :  136.12797045707703
166228 :  136.1339569091797
166229 :  136.1439278125763
166231 :  136.14991402626038
166236 :  136.15590000152588
166242 :  136.16184997558594
166246 :  136.16786646842957
166248 :  136.17385053634644
166249 :  136.18083000183105
166262 :  136.1858115196228
166263 :  136.19180297851562
166266 :  136.19778609275818
166269 :  136.20377159118652
166273 :  136.2097589969635
166276 :  136.21573781967163
166277 :  136.2217297554016
166292 :  136.22771191596985
166293 :  136.23369121551514
166294 :  136.2386782169342
166307 :  136.24466180801392
166309 :  136.250648021698
166313 :  136.25663685798645
166315 :  136.26261448860168
166316 :  136.26859736442566
166317 :  136.27458453178406
166318 :  136.28056716918945
166320 :  136.28655076026917
166327 :  136.29253125190735
166328 :  136.2985155582428
166330 :  136.30549669265747
166331 :  136.310452222

167391 :  137.97364139556885
167396 :  137.97962498664856
167401 :  137.9856095314026
167406 :  137.9915931224823
167408 :  137.9975769519806
167415 :  138.00359201431274
167417 :  138.00957655906677
167418 :  138.0165593624115
167419 :  138.02254152297974
167420 :  138.02752780914307
167422 :  138.0335111618042
167426 :  138.03949546813965
167429 :  138.0454876422882
167430 :  138.0514633655548
167431 :  138.05744862556458
167432 :  138.06343245506287
167434 :  138.06941843032837
167439 :  138.07540035247803
167451 :  138.08138418197632
167455 :  138.0873682498932
167457 :  138.09335017204285
167461 :  138.09933590888977
167466 :  138.10531997680664
167472 :  138.11130380630493
167473 :  138.11728620529175
167475 :  138.1232717037201
167481 :  138.12826132774353
167485 :  138.1342430114746
167486 :  138.14022707939148
167489 :  138.14621925354004
167490 :  138.1521978378296
167491 :  138.15818238258362
167493 :  138.16512966156006
167495 :  138.1721112728119
167496 :  138.178095817565

In [19]:
for i in IDs:
    print(i)

1
6
10
14
16
18
20
23
26
30
37
41
49
75
78
80
85
89
93
95
98
104
107
114
116
119
120
131
134
135
138
139
143
147
149
155
157
158
170
173
174
175
186
193
194
196
197
198
200
201
203
204
205
207
213
214
216
217
223
225
226
227
231
233
234
237
238
240
247
249
260
262
264
266
267
268
272
273
277
282
283
289
292
293
304
321
325
327
334
335
336
342
343
347
350
360
363
364
365
366
367
369
372
376
377
379
381
387
389
392
394
399
405
415
419
422
423
425
427
434
441
453
455
456
467
480
483
487
488
502
503
504
506
507
510
511
516
518
519
523
526
534
536
537
542
543
550
552
553
558
562
563
567
571
576
579
588
595
603
610
613
617
619
620
633
635
653
656
660
665
669
681
683
690
692
695
712
713
720
722
723
724
727
729
731
739
740
744
746
747
749
755
760
770
771
777
780
782
791
792
794
805
817
819
821
824
831
832
852
855
856
862
864
870
871
872
874
876
878
879
882
887
888
889
892
893
903
905
908
910
913
917
918
922
931
932
935
937
938
939
945
947
948
949
955
963
972
976
978
981
984
985
989
995
998
999

8381
8390
8391
8401
8405
8407
8409
8417
8419
8420
8429
8430
8434
8435
8437
8439
8441
8442
8446
8449
8453
8459
8461
8465
8468
8473
8474
8477
8479
8483
8489
8490
8491
8493
8499
8504
8524
8525
8529
8536
8543
8545
8546
8554
8560
8561
8570
8577
8582
8584
8590
8592
8597
8603
8605
8617
8621
8624
8629
8630
8633
8635
8637
8640
8648
8649
8651
8655
8657
8658
8660
8662
8664
8669
8670
8671
8678
8681
8682
8684
8689
8691
8696
8699
8700
8701
8702
8709
8710
8715
8716
8728
8730
8732
8733
8742
8754
8759
8760
8764
8769
8772
8775
8787
8788
8789
8794
8802
8807
8809
8824
8826
8839
8844
8849
8851
8852
8853
8858
8861
8862
8863
8866
8869
8877
8879
8884
8891
8896
8900
8907
8909
8912
8927
8929
8933
8938
8941
8942
8946
8947
8949
8954
8956
8960
8962
8963
8971
8975
8976
8977
8990
8998
9001
9002
9006
9010
9011
9015
9019
9021
9023
9025
9031
9035
9041
9042
9046
9047
9051
9061
9065
9068
9071
9072
9074
9077
9080
9081
9088
9091
9092
9093
9095
9098
9100
9101
9117
9130
9133
9136
9137
9143
9147
9153
9158
9163
9164
9166
9170


16451
16452
16457
16461
16463
16464
16470
16476
16480
16488
16489
16491
16493
16495
16508
16512
16515
16518
16519
16524
16533
16541
16558
16563
16564
16565
16566
16567
16571
16574
16577
16579
16584
16588
16589
16590
16598
16602
16605
16610
16613
16616
16619
16628
16629
16630
16635
16637
16642
16647
16651
16652
16654
16658
16665
16668
16671
16672
16673
16675
16679
16681
16683
16687
16690
16691
16694
16698
16704
16705
16707
16715
16720
16723
16724
16725
16728
16734
16740
16742
16748
16752
16755
16758
16769
16770
16771
16774
16775
16776
16781
16785
16789
16794
16795
16796
16797
16801
16802
16804
16805
16812
16814
16815
16816
16817
16820
16827
16828
16829
16831
16833
16839
16843
16851
16853
16855
16856
16857
16858
16861
16863
16864
16870
16872
16877
16879
16882
16884
16886
16892
16894
16898
16899
16901
16902
16905
16906
16908
16911
16912
16915
16916
16917
16927
16930
16932
16935
16936
16937
16941
16942
16943
16949
16950
16953
16955
16956
16957
16958
16961
16962
16964
16968
16979
16981
1698

29096
29099
29100
29114
29118
29123
29127
29149
29150
29152
29153
29163
29173
29176
29177
29186
29188
29192
29204
29209
29214
29218
29227
29235
29246
29255
29257
29258
29271
29275
29292
29295
29297
29315
29316
29318
29327
29335
29347
29351
29357
29361
29375
29377
29383
29403
29405
29413
29414
29421
29423
29428
29429
29440
29445
29450
29452
29461
29463
29465
29470
29473
29477
29483
29484
29492
29515
29517
29518
29522
29527
29531
29540
29541
29545
29560
29563
29566
29568
29574
29585
29588
29594
29596
29597
29601
29610
29612
29616
29617
29619
29622
29625
29626
29628
29631
29632
29638
29641
29650
29656
29659
29665
29668
29671
29676
29684
29688
29697
29698
29703
29705
29707
29711
29715
29724
29732
29739
29741
29770
29777
29783
29793
29826
29830
29879
29884
29889
29902
29906
29908
29919
29922
29928
29945
29948
29952
29961
29975
29977
29988
29996
30003
30004
30015
30016
30023
30026
30060
30062
30067
30071
30076
30079
30092
30095
30100
30104
30112
30114
30119
30120
30137
30150
30157
30158
3016

51500
51510
51525
51536
51556
51560
51577
51604
51614
51630
51678
51718
51721
51743
51754
51758
51764
51765
51791
51792
51794
51820
51834
51843
51852
51853
51883
51885
51904
51913
51914
51918
51919
51930
51933
51946
51953
51954
51963
51967
51968
51983
52011
52038
52053
52081
52083
52095
52096
52122
52127
52138
52148
52154
52156
52158
52163
52178
52181
52193
52194
52201
52203
52218
52229
52230
52238
52239
52241
52260
52272
52285
52286
52295
52299
52316
52340
52347
52348
52358
52368
52388
52396
52405
52417
52422
52445
52456
52506
52519
52524
52527
52528
52532
52549
52559
52564
52566
52568
52571
52576
52584
52588
52589
52602
52607
52612
52613
52622
52629
52645
52649
52667
52677
52680
52693
52710
52711
52723
52725
52726
52728
52729
52734
52741
52747
52752
52766
52771
52772
52777
52793
52813
52827
52830
52839
52840
52846
52847
52871
52873
52880
52882
52924
52925
52938
52939
52946
52951
52968
52972
52982
53002
53006
53034
53035
53041
53045
53062
53071
53098
53120
53122
53145
53146
53148
5315

...

98488
98504
98505
98511
98518
98531
98532
98538
98539
98542
98545
98555
98561
98578
98593
98616
98627
98644
98665
98687
98698
98711
98732
98735
98744
98753
98780
98784
98786
98801
98817
98827
98828
98831
98851
98855
98856
98859
98860
98876
98897
98899
98905
98906
98913
98941
98942
98967
98972
98976
98991
99018
99019
99027
99028
99029
99050
99052
99056
99058
99059
99060
99077
99100
99102
99108
99133
99155
99185
99202
99211
99231
99236
99258
99259
99274
99281
99284
99297
99315
99333
99338
99340
99366
99374
99375
99382
99384
99391
99403
99413
99421
99452
99457
99460
99481
99511
99516
99523
99528
99561
99562
99596
99607
99608
99622
99623
99629
99631
99668
99669
99672
99710
99717
99721
99726
99727
99728
99750
99760
99764
99765
99772
99774
99799
99800
99803
99826
99875
99876
99879
99889
99898
99899
99911
99912
99920
99933
99946
99960
99970
99981
99996
100007
100009
100011
100014
100048
100054
100064
100083
100086
100088
100089
100099
100101
100110
100112
100128
100144
100156
100158
100182
10

120169
120221
120226
120230
120232
120251
120262
120283
120284
120288
120299
120317
120318
120328
120331
120351
120375
120425
120426
120429
120430
120458
120465
120496
120519
120534
120535
120536
120542
120549
120553
120557
120558
120562
120574
120575
120582
120583
120617
120622
120628
120641
120643
120653
120654
120657
120658
120666
120672
120685
120686
120697
120715
120721
120725
120729
120732
120738
120763
120764
120783
120836
120841
120862
120864
120868
120872
120881
120882
120893
120933
120958
120964
120967
120985
120998
121007
121060
121061
121077
121079
121087
121091
121096
121109
121116
121121
121122
121125
121128
121143
121150
121167
121209
121226
121245
121247
121256
121258
121260
121278
121283
121294
121300
121314
121334
121335
121343
121363
121365
121389
121392
121398
121417
121425
121447
121455
121465
121473
121474
121475
121479
121483
121484
121492
121505
121524
121525
121528
121529
121532
121542
121558
121590
121606
121607
121625
121630
121631
121659
121667
121672
121709

143567
143579
143581
143582
143596
143642
143655
143660
143664
143670
143680
143691
143716
143719
143720
143728
143788
143794
143801
143805
143811
143825
143828
143829
143846
143848
143853
143854
143871
143896
143932
143950
143964
143983
143993
144030
144079
144085
144120
144130
144138
144142
144204
144215
144261
144267
144316
144329
144335
144351
144370
144378
144388
144440
144442
144448
144497
144523
144529
144535
144541
144549
144579
144586
144604
144611
144625
144655
144683
144695
144738
144759
144773
144859
144875
144912
144923
144953
144974
144975
144979
144989
144998
145019
145026
145039
145048
145053
145055
145073
145079
145098
145124
145138
145183
145186
145198
145204
145252
145256
145266
145268
145270
145275
145286
145289
145313
145344
145365
145369
145392
145403
145415
145431
145441
145447
145497
145517
145518
145557
145580
145594
145601
145603
145650
145666
145693
145734
145753
145754
145758
145760
145771
145777
145787
145792
145803
145806
145818
145852
145857
145872
145880

159373
159375
159382
159391
159393
159395
159398
159399
159411
159417
159427
159429
159432
159433
159434
159437
159440
159445
159448
159449
159452
159456
159457
159459
159464
159465
159474
159476
159477
159486
159487
159488
159493
159498
159499
159501
159506
159508
159509
159512
159513
159518
159519
159521
159522
159525
159534
159538
159539
159542
159546
159548
159555
159561
159563
159565
159569
159570
159572
159573
159575
159580
159582
159588
159590
159596
159601
159602
159603
159611
159614
159616
159617
159618
159632
159633
159641
159642
159644
159645
159651
159657
159662
159663
159665
159673
159674
159680
159682
159691
159693
159708
159714
159716
159717
159721
159722
159727
159731
159732
159736
159737
159741
159747
159759
159762
159767
159770
159771
159773
159774
159783
159791
159793
159804
159805
159806
159808
159818
159819
159820
159824
159826
159827
159829
159836
159842
159843
159855
159861
159864
159865
159868
159875
159879
159884
159892
159897
159899
159900
159903
159909
159912

167490
167491
167493
167495
167496
167497
167499
167501
167503
167504
167505
167507
167509
167510
167514
167515
167529
167536
167547
167550
167551
167552
167557
167558
167561
167564
167570
167571
167572
167581
167583
167593
167594
167596
167598
167600
167602
167603
167604
167608
167612
167616
167618
167621
167625
167634
167639
167640
167645
167647
167648
167649
167650
167662
167673
167675
167680
167681
167682
167697
167698
167700
167704
167707
167709
167712
167715
167722
167724
167725
167727
167729
167731
167734
167735
167736
167738
167741
167743
167745
167748
167749
167751
167752
167754
167762
167764
167776
167785
167787
167791
167792
167796
167805
167813
167816
167819
167821
167822
167823
167824
167825
167831
167835
167838
167842
167845
167849
167860
167866
167872
167878
167885
167888
167890
167894
167898
167903
167904
167908
167909
167910
167912
167914
167916
167917
167924
167930
167931
167946
167948
167953
167955
167958
167970
167972
167978
167980
167982
167988
167991
168002
168006

In [9]:
userGroups = (snapData.groupby(["VEHICLE"]))
userGroups.groups.keys()
IDs = [1, 6, 10, 14, 16, 18, 20, 23, 26, 30, 37, 41, 49, 75, 78, 80, 85, 89, 93, 95, 98, 104, 107, 114, 116, 119, 120, 131, 134, 135, 138, 139, 143, 147, 149, 155, 157, 158, 170, 173, 174, 175, 186, 193, 194, 196, 197, 198, 200, 201, 203, 204, 205, 207, 213, 214, 216, 217, 223, 225, 226, 227, 231, 233, 234, 237, 238, 240, 247, 249, 260, 262, 264, 266, 267, 268, 272, 273, 277, 282, 283, 289, 292, 293, 304, 321, 325, 327, 334, 335, 336, 342, 343, 347, 350, 360, 363, 364, 365, 366, 367, 369, 372, 376, 377, 379, 381, 387, 389, 392, 394, 399, 405, 415, 419, 422, 423, 425, 427, 434, 441, 453, 455, 456, 467, 480, 483, 48]
IDs = IDs[0:100]
IDs


[1,
 6,
 10,
 14,
 16,
 18,
 20,
 23,
 26,
 30,
 37,
 41,
 49,
 75,
 78,
 80,
 85,
 89,
 93,
 95,
 98,
 104,
 107,
 114,
 116,
 119,
 120,
 131,
 134,
 135,
 138,
 139,
 143,
 147,
 149,
 155,
 157,
 158,
 170,
 173,
 174,
 175,
 186,
 193,
 194,
 196,
 197,
 198,
 200,
 201,
 203,
 204,
 205,
 207,
 213,
 214,
 216,
 217,
 223,
 225,
 226,
 227,
 231,
 233,
 234,
 237,
 238,
 240,
 247,
 249,
 260,
 262,
 264,
 266,
 267,
 268,
 272,
 273,
 277,
 282,
 283,
 289,
 292,
 293,
 304,
 321,
 325,
 327,
 334,
 335,
 336,
 342,
 343,
 347,
 350,
 360,
 363,
 364,
 365,
 366]

In [10]:
# def completeRecords(snapData):#snapData is a dataframe
#     userGroups = (snapData.groupby(["VEHICLE"]))
#     concatData = {}
#     for ID in userGroups.groups.keys():
#         records.TIME = pd.to_timedelta(records.TIME, unit="s")
#         records.set_index(["TIME"],inplace=True)
#         records.loc[records.index[0]+pd.to_timedelta('24:00:00')]=records.iloc[0,:]
#         records.sort_index(inplace=True)
#         records = records.resample('30S').fillna("pad")
#         records.drop([records.index[0]+pd.to_timedelta('24:00:00')],axis=0,inplace=True)
#         concatData[ID] = kk
#     return concatData

In [5]:
# #we time the process
# startTime = time.time()
# concatData = pd.DataFrame()
# userGroups = (snapData.groupby(["VEHICLE"]))
# IDs = [1,6,30,100007,100009,10001,100011,100014,100048,100054]
# futures = []
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:#max_workers=7
# #     for ID in IDs:#userGroups.groups.keys(),IDs
#         futures.append(executor.map(completeRecords, snapData)) #records=userGroups.get_group(ID)
#     for future in concurrent.futures.as_completed(futures):
#         concatData = concatData.append(future.result())
        
        
# # futures = np.empty([])       
# # with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:#max_workers=7
# #     for ID in IDs:#userGroups.groups.keys(),IDs
        
# #         executor.map(completeRecords, records=userGroups.get_group(ID))
        
# #         futures[index] = 
# #     for future in concurrent.futures.as_completed(futures):
# #         concatData = concatData.append(future.result())
        

        
        
#  demand = np.empty([len(addresses),18])
# with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
#     for index, threadDemand in zip(addresses.keys(),executor.map(demandSupply,addresses.values())):
#         demand[index,] = threadDemand   
    
    
    
        
# # with open("completePLUdata_30sec.pickle".format(a=savingLoc),"wb") as handle:
# #           pickle.dump(concatData,handle, protocol=pickle.HIGHEST_PROTOCOL )
# print(concatData)
# print((time.time() - startTime)//60,'minutes')

TypeError: map() got an unexpected keyword argument 'records'